## Final Term Project Milestone 4
**April Meyer  
DSC540-T301 Data Preparation (2207-1)**

	1. Perform at least 5 data transformation and/or cleansing steps to your Connecting to an API/Pulling in the Data and Cleaning/Formatting.
        • Replace Headers
        • Format data into a more readable format
        • Identify outliers and bad data
        • Find duplicates
        • Fix casing or inconsistent values
        • Conduct Fuzzy Matching
        
        
**My goal here is to use this API and search function in the last milestone. I want to take the authors from the best seller lists and find their other work.**

In [3]:
# Import the necessary libraries
import urllib.request, urllib.parse, urllib.error
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from string import digits 
import json

In [5]:
search = 'Delia+Owens'
# Opening my the APIkeys.json file to retrieve my Goodreads Key
with open('APIkeys.json') as f:
    keys = json.loads(f.read())
    GoodReads = keys[0]['GoodReads']
# Assigning the Goodreads url as a string to a variable. 
serviceurl = 'https://www.goodreads.com/search/index.xml?'
apikey = '&key='+ GoodReads
url = serviceurl + apikey + search
print(url)

https://www.goodreads.com/search/index.xml?&key=dZJ8HTjP544O1EKH0MIGQDelia+Owens


In [131]:
def to_dataframe(list):
    # 1. Format data into a more readable format from list to dataframe
    additional_booksby_author = pd.DataFrame.from_records(to_dataframe,columns=['Book Name','Author','Average_Rating'])
    #2. Clean Authors Name - remove newline
    additional_booksby_author['Author'].replace('\n', '', regex=True, inplace=True)
    #3. Clean Authors Name - remove numbers
    additional_booksby_author['Author'] = additional_booksby_author['Author'].str.replace('\d+', '')
    return additional_booksby_author


In [132]:
alls = []
#additional_booksby_author = []
def search_author(author):
    search = '&q=' + author
    url = serviceurl + apikey + search
    uh = urllib.request.urlopen(url)
    data = uh.read()
    soup = BeautifulSoup(contents,'xml')
    for d in soup.findAll('work'):
        titles = d.find('title')
        authors = d.find('author')
        average_rating = d.find('average_rating')
        all1=[]
        if titles is not None:
            all1.append(titles.get_text())
        else:
            all1.append("unknown-product")

        if author is not None:
            all1.append(authors.get_text())
        else:
            all1.append("unknown-author")

        if average_rating is not None:
            all1.append(average_rating.get_text())
        else:
            all1.append("unknown-rating")

        alls.append(all1)
    # 1. Format data into a more readable format from list to dataframe
    additional_booksby_author = pd.DataFrame.from_records(alls,columns=['Book Name','Author','Average_Rating'])
    #2. Clean Authors Name - remove newline
    additional_booksby_author['Author'].replace('\n', '', regex=True, inplace=True)
    #3. Clean Authors Name - remove numbers
    additional_booksby_author['Author'] = additional_booksby_author['Author'].str.replace('\d+', '')
    #4. Replacing the + from the search with a space
    author = author.replace('+', ' ')
    #5. Removing books that are not by searched author
    # Get names of indexes for which column is not the search author
    indexNames = additional_booksby_author[ additional_booksby_author['Author'] != author].index
    # Delete these row indexes from dataFrame
    additional_booksby_author.drop(indexNames , inplace=True)    
    #print(author_books)
    return additional_booksby_author  

author_df = search_author("Delia+Owens")
author_df

,Book Name,Author,Average_Rating
0,Where the Crawdads Sing,Delia Owens,4.48
3,The Eye of the Elephant: An Epic Adventure in ...,Delia Owens,4.22
